<a href="https://colab.research.google.com/github/ggoddll99/ds_study/blob/main/240906_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **분류 연습 문제**
___
출처 : 핸즈온 머신러닝 Ch03 분류 연습문제 1, 2번

In [1]:
# import data
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version = 1, as_frame = False, parser='auto')

In [2]:
X, y = mnist["data"], mnist["target"]

In [3]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

### **1. MNIST 데이터셋으로 분류기를 만들어 테스트 세트에서 97% 정확도를 달성해보세요.**
___

1. `KNeghtborsClassifier`를 사용하는 것을 추천합니다.
2. `weights`와 `n_neighbors` 하이퍼 파라미터로 그리드 탐색을 시도하여, 좋은 하이퍼 파라미터 값을 찾아보세요.

In [4]:
# import package
from sklearn.neighbors import KNeighborsClassifier

In [5]:
# Try GridSearch to optimize hyperparameter
knn_clf=KNeighborsClassifier()
knn_clf.fit(X_train, y_train)

KNeighborsClassifier()

In [6]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'weights': ['uniform', 'distance'],
    'n_neighbors': [1, 2, 3]
}

gsCV = GridSearchCV(knn_clf, param_grid, cv=5,
                                scoring='accuracy', n_jobs=-1)
gsCV.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': [1, 2, 3],
                         'weights': ['uniform', 'distance']},
             scoring='accuracy')

In [7]:
# best hyperparameter
print("Best parameters found:", gsCV.best_params_)

Best parameters found: {'n_neighbors': 3, 'weights': 'distance'}


In [8]:
# best score
print("Best cross-validation score:", gsCV.best_score_)

Best cross-validation score: 0.9711166666666665


In [9]:
# model test
from sklearn.metrics import accuracy_score, classification_report

# 1. 테스트 데이터로 모델 예측
y_pred = gsCV.best_estimator_.predict(X_test)

# 2. 정확도(accuracy) 평가
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

# 3. 상세한 성능 평가 (Precision, Recall, F1-score)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Test Accuracy: 0.9717
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       980
           1       0.97      1.00      0.98      1135
           2       0.98      0.97      0.98      1032
           3       0.97      0.97      0.97      1010
           4       0.98      0.97      0.97       982
           5       0.96      0.96      0.96       892
           6       0.98      0.99      0.98       958
           7       0.96      0.97      0.96      1028
           8       0.99      0.95      0.97       974
           9       0.96      0.96      0.96      1009

    accuracy                           0.97     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.97      0.97      0.97     10000



### **2. 다음 단계를 따라 인위적으로 훈련 세트를 늘리는 데이터 증식 또는 훈련 세트 확장 기법을 연습해봅시다.**
___

#### **STEP 1. MNIST 이미지를 (왼, 오른, 위, 아래) 어느 방향으로든 한 픽셀 이동시킬 수 있는 함수를 만들어 보세요.**

In [10]:
import numpy as np
from scipy.ndimage import shift

def shift_mnist_image(image, direction):
    # 이미지가 1D 배열로 들어오면 28x28 크기의 2D 배열로 변환
    image = image.reshape((28, 28))

    # 방향에 따른 이동량 설정
    if direction == 'left':
        shifted_image = shift(image, [0, -1], cval=0, mode='constant')
    elif direction == 'right':
        shifted_image = shift(image, [0, 1], cval=0, mode='constant')
    elif direction == 'up':
        shifted_image = shift(image, [-1, 0], cval=0, mode='constant')
    elif direction == 'down':
        shifted_image = shift(image, [1, 0], cval=0, mode='constant')

    # 이동된 이미지를 1D 배열로 변환하여 반환
    return shifted_image.reshape([-1])

####  **STEP 2. 앞에서 만든 함수를 이용하여, 훈련 세트에 있는 각 이미지에 대해 네 개의 이동된 복사본(방향마다 한 개씩)을 만들어 훈련 세트에 추가하세요**

In [11]:
augmented_images = []
augmented_labels = []

for i in range(len(X_train)):
    # 원본 이미지와 레이블 추가
    augmented_images.append(X_train[i])
    augmented_labels.append(y_train[i])

    # 이동된 이미지 추가
    for direction in ['left', 'right', 'up', 'down']:
        shifted_image = shift_mnist_image(X_train[i], direction)
        augmented_images.append(shifted_image)
        augmented_labels.append(y_train[i])  # 레이블은 원본과 동일하게 추가

# 3. 증강된 데이터셋을 numpy 배열로 변환
X_train_augmented = np.array(augmented_images)
y_train_augmented = np.array(augmented_labels)

# 데이터셋 크기 확인
print(f"원본 훈련 데이터 크기: {X_train.shape}")
print(f"증강된 훈련 데이터 크기: {X_train_augmented.shape}")

원본 훈련 데이터 크기: (60000, 784)
증강된 훈련 데이터 크기: (300000, 784)


####  **STEP 3. 위에서 확장한 데이터셋을 이용하여, 1번 문제에서 찾은 최적 모델을 훈련시키고, 테스트 세트에서 정확도를 측정해보세요**

In [12]:
knn_clf = KNeighborsClassifier(n_neighbors = 3, weights = "distance")
knn_clf.fit(X_train_augmented, y_train_augmented)

y_pred = knn_clf.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.9763
